# Code 1 - SQL Agent 
- Open API
- Langchain
- sqlite database

### Referances
- sqlite (download portable package) https://sqlitebrowser.org/dl/

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create Connection URL

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)

### Check Databases 
- if the database is blank, please use sqlite browser to populate the same

In [3]:
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [4]:
print(db.table_info)


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Empl

### Connect to sqlite database

In [5]:
con = connect("chinook.db")

### Check Sample Query

In [6]:
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")

In [7]:
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Load Environment Variables 
- Mostly API Keys

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
API_KEY = os.getenv("GROQ_API_KEY")

### Load Packages

In [12]:
from langchain.callbacks import get_openai_callback
from langchain_groq import ChatGroq

### Create an Instance of LLM with configuration

In [13]:
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)

### Check LLM 

In [14]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DC53FC2690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DC54C66110>, temperature=1e-08, groq_api_key=SecretStr('**********'))

## Step 3 - Create Chain

### Import Packages

In [15]:
from langchain_experimental.sql.base import SQLDatabaseChain

### Setup SQL DB Chain

In [16]:
sql_db_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True)

### Run and Print Chain Output

In [17]:
result = sql_db_chain.invoke("Which country's customers spent the most?")
print(result)



> Entering new SQLDatabaseChain chain...
Which country's customers spent the most?
SQLQuery:SELECT Country, SUM(Total) as TotalSpent
FROM Invoice
JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
GROUP BY Country
ORDER BY TotalSpent DESC
LIMIT 5;
SQLResult: [('USA', 523.06), ('Canada', 303.96), ('France', 195.1), ('Brazil', 190.1), ('Germany', 156.48)]
Answer:Customers from the USA spent the most, with a total of 523.06.
> Finished chain.
{'query': "Which country's customers spent the most?", 'result': 'Customers from the USA spent the most, with a total of 523.06.'}


### Run and Print Output with Callback
- Get # Tokens used
- Get # Requests
- Get # Completion Tokens

In [18]:
with get_openai_callback() as cb:
    result = sql_db_chain.invoke("Which country's customers spent the most?")
print("LLM Result: ", result['result'])
print(cb)



> Entering new SQLDatabaseChain chain...
Which country's customers spent the most?
SQLQuery:SELECT Country, SUM(Total) as TotalSpent
FROM Invoice
JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
GROUP BY Country
ORDER BY TotalSpent DESC
LIMIT 5;
SQLResult: [('USA', 523.06), ('Canada', 303.96), ('France', 195.1), ('Brazil', 190.1), ('Germany', 156.48)]
Answer:Customers from the USA spent the most, with a total of 523.06.
> Finished chain.
LLM Result:  Customers from the USA spent the most, with a total of 523.06.
Tokens Used: 6624
	Prompt Tokens: 6492
	Completion Tokens: 132
Successful Requests: 2
Total Cost (USD): $0.0


In [19]:
with get_openai_callback() as cb:
    result = sql_db_chain.invoke("List the total sales per country. Which country's customers spent the most?")
print("LLM Result: ", result['result'])
print(cb)



> Entering new SQLDatabaseChain chain...
List the total sales per country. Which country's customers spent the most?
SQLQuery:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j3s8p5twe97rfp2ktsdx1bvm` on tokens per minute (TPM): Limit 5000, Used 6861, Requested 1790. Please try again in 43.818s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

### Check Template

In [ ]:
print(sql_db_chain.llm_chain.prompt.template)

# END - Next Topic - SQL Agents